In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
import time

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import torch
torch.set_printoptions(edgeitems=2, linewidth=75)
torch.manual_seed(123)

In [ ]:
#Importing the GPU and code is ran to check if gpu is available and what GPU

gpu_available = torch.cuda.is_available()
print(f"Is the GPU available? {gpu_available}")

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("Device", device)

Is the GPU available? True
Device cuda


In [ ]:
#Calculate the CPU and GPU timing as we can guess that the GPU will be quicker
x = torch.randn(5000, 5000)
startTime = time.time()
_ = torch.matmul(x, x)
endTime = time.time()
print(f"CPU time: {(endTime - startTime):6.5f}s")
x = x.to(device)
start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)
start.record()
_ = torch.matmul(x, x)
end.record()
torch.cuda.synchronize()
print(f"GPU time: {0.001 * start.elapsed_time(end):6.5f}s")

CPU time: 2.08234s
GPU time: 2.86504s


In [ ]:
##Creating a datapath and transforming to normalize
from torch.utils import data
from torchvision import datasets
from torchvision import transforms
import torch.utils.data as data

data_path = '../data'
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))])

train_dataset = datasets.CIFAR10(data_path, train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(data_path, train=False, download=True, transform=transform)

data_labels = ('plane', 'car', 'bird', 'cat', )

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
    torch.backends.cudnn.determinstic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
# Load CIFAR-10 dataset
(trainImages, trainLabels), (testImages, testLabels) = cifar10.load_data()

# Normalize pixel values to be between 0 and 1
trainImages = trainImages / 255.0
testImages = testImages / 255.0

# Convert labels to one-hot encoding
trainLabels = to_categorical(trainLabels)
testLabels = to_categorical(testLabels)

# Build the convolutional neural network (CNN) model
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Record the start time for training
startTime = time.time()

# Train the model
history = model.fit(trainImages, trainLabels, epochs=300, validation_data=(testImages, testLabels))

# Record the end time for training
endTime = time.time()

# Print the training time
print(f"Training Time: {endTime - startTime} seconds")

# Retrieve the final training loss
training_loss = history.history['loss'][-1]

# Evaluate the model on the test set and print the accuracy
evaluation_accuracy = model.evaluate(testImages, testLabels)[1]
print(f"Training Loss: {training_loss}")
print(f"Evaluation Accuracy: {evaluation_accuracy}")

Epoch 1/300
1563/1563 [==============================] - 19s 6ms/step - loss: 1.4901 - accuracy: 0.4586 - val_loss: 1.2449 - val_accuracy: 0.5580
Epoch 2/300
1563/1563 [==============================] - 10s 6ms/step - loss: 1.1268 - accuracy: 0.6016 - val_loss: 1.0851 - val_accuracy: 0.6156
Epoch 3/300
1563/1563 [==============================] - 10s 6ms/step - loss: 0.9733 - accuracy: 0.6574 - val_loss: 1.0635 - val_accuracy: 0.6253
Epoch 4/300
1563/1563 [==============================] - 10s 7ms/step - loss: 0.8751 - accuracy: 0.6926 - val_loss: 0.9118 - val_accuracy: 0.6867
Epoch 5/300
1563/1563 [==============================] - 11s 7ms/step - loss: 0.7923 - accuracy: 0.7222 - val_loss: 0.8745 - val_accuracy: 0.6971
Epoch 6/300
1563/1563 [==============================] - 11s 7ms/step - loss: 0.7275 - accuracy: 0.7465 - val_loss: 0.8966 - val_accuracy: 0.6960
Epoch 7/300
1563/1563 [==============================] - 10s 6ms/step - loss: 0.6725 - accuracy: 0.7639 - val_loss: 0.8871 -

In [ ]:
model_extended = models.Sequential()
model_extended.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model_extended.add(layers.MaxPooling2D((2, 2)))
model_extended.add(layers.Conv2D(64, (3, 3), activation='relu'))
model_extended.add(layers.MaxPooling2D((2, 2)))
model_extended.add(layers.Conv2D(128, (3, 3), activation='relu'))
model_extended.add(layers.MaxPooling2D((2, 2)))
model_extended.add(layers.Flatten())
model_extended.add(layers.Dense(256, activation='relu'))
model_extended.add(layers.Dense(10, activation='softmax'))

model_extended.compile(optimizer='adam',
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])

start_time_extended = time.time()
history_extended = model_extended.fit(trainImages, trainLabels, epochs=300, validation_data=(testImages, testLabels))
end_time_extended = time.time()

print(f"Extended Model Training Time: {end_time_extended - start_time_extended} seconds")
print(f"Extended Model Training Loss: {history_extended.history['loss'][-1]}")
print(f"Extended Model Evaluation Accuracy: {model_extended.evaluate(testImages, testLabels)[1]}")

Epoch 1/300
1563/1563 [==============================] - 12s 6ms/step - loss: 1.4886 - accuracy: 0.4594 - val_loss: 1.2282 - val_accuracy: 0.5730
Epoch 2/300
1563/1563 [==============================] - 9s 6ms/step - loss: 1.0980 - accuracy: 0.6133 - val_loss: 1.0242 - val_accuracy: 0.6395
Epoch 3/300
1563/1563 [==============================] - 10s 6ms/step - loss: 0.9346 - accuracy: 0.6715 - val_loss: 1.0269 - val_accuracy: 0.6449
Epoch 4/300
1563/1563 [==============================] - 10s 6ms/step - loss: 0.8223 - accuracy: 0.7121 - val_loss: 0.9027 - val_accuracy: 0.6858
Epoch 5/300
1563/1563 [==============================] - 9s 6ms/step - loss: 0.7319 - accuracy: 0.7446 - val_loss: 0.8501 - val_accuracy: 0.7062
Epoch 6/300
1563/1563 [==============================] - 10s 6ms/step - loss: 0.6625 - accuracy: 0.7684 - val_loss: 0.8378 - val_accuracy: 0.7096
Epoch 7/300
1563/1563 [==============================] - 10s 6ms/step - loss: 0.6005 - accuracy: 0.7894 - val_loss: 0.8236 - v